# Введение

**ReAct** (`Reasoning + Acting`) — это подход, при котором **LLM** чередует размышления (`Thought`) и действия (`Action`), прежде чем выдать окончательный ответ.

**Структура ответа:**

```text
Question: В каком году был основан Google?
Thought: Нужно ли использовать инструмент? (Yes)
Action: Search
Action Input: В каком году был основан Google?
Observation: Google был основан в 1998 году.
Thought: У меня достаточно информации.
Final Answer: Google был основан в 1998 году.
```



# Установка зависимостей

```text
langchain ------------> Работа с LLM моделями
langchain_community --> Компоненты сообщества (инструменты, модели и т.п.)
transformers ---------> Загрузка и работа с моделями из Hugging Face
torch ----------------> Фреймворк для работы с тензорами и моделями
bitsandbytes ---------> Оптимизация памяти
duckduckgo-search ----> Поиск в интернете без API-ключа
accelerate -----------> Оптимизация запуска модели


```


In [ ]:
# Установка всех необходимых библиотек
!pip install -q langchain langchain_community transformers duckduckgo-search torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Проверка установки
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from huggingface_hub import login
from langchain_community.llms import HuggingFacePipeline
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

print("✅ Все библиотеки импортированы успешно!")

✅ Все библиотеки импортированы успешно!


# Подключение и загрузка модели


Модель Qwen2.5-7B-Instruct от Alibaba и компоненты, с которыми будет запускаться модель такие:
* **AutoTokenizer** — преобразует текст в токены
* **AutoModelForCausalLM** — модель генерации текста
* **pipeline** — упрощает вызов модели


Авторизуемся в [Hugging Face](https://huggingface.co/)

In [ ]:
import os
from huggingface_hub import login

login(token=os.getenv("hf"))

Загрузка модели и токенизатора

In [ ]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Создание пайплайна генерации текста

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.1,
    max_new_tokens=1024,
    do_sample=False,        # Жадная генерация
    return_full_text=False  # Возвращать только сгенерированную часть
)

print("✅ Модель загружена и пайплайн готов!")

Device set to use cuda:0


✅ Модель загружена и пайплайн готов!


# Интеграция модели в LangChain через HuggingFacePipeline


Обёртка модели в LangChain

In [ ]:
from langchain_community.llms import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "stop": ["\nObservation:"]
    }
)

print("✅ Модель интегрирована в LangChain")

✅ Модель интегрирована в LangChain


# Добавление инструмента поиска (DuckDuckGo)


In [ ]:
!pip install -U ddgs --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import Tool

search_tool = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="Инструмент поиска"
    )
]

print("✅ Инструмент поиска добавлен")

✅ Инструмент поиска добавлен


# Создание ReAct-промпта


In [ ]:
from langchain.prompts import PromptTemplate

react_prompt_template = """
You are an intelligent agent that strictly follows the ReAct (Reasoning + Acting) pattern.

You can use these tools: {tool_names}

Respond ONLY using the format below:

Question: the input question you must answer
Thought: Do I need to use a tool? (Yes or No)
Action: the name of the tool to use, exactly one of [{tool_names}, {tools}]
Action Input: the input to the tool
Observation: the result of the action
Thought: I have enough information
Final Answer: the final answer to the original input question

Example:

Question: What is the capital of France?
Thought: Yes
Action: Search
Action Input: Capital of France
Observation: The capital of France is Paris.
Thought: I have enough information
Final Answer: The capital of France is Paris.

RULES:
- Do NOT write anything outside the format.
- ALWAYS follow the format exactly.
- Do NOT skip any fields.
- NEVER write Final Answer before Observation.
- If no tool is needed, write Action: None and Observation: None.

Begin!

Question: {input}
{agent_scratchpad}
"""

# Инициализация PromptTemplate
prompt = PromptTemplate(
    template=react_prompt_template,
    input_variables=["input", "agent_scratchpad", "tool_names"]
).partial(
    tool_names=", ".join([tool.name for tool in tools])
)

print("✅ ReAct-промпт создан")

✅ ReAct-промпт создан


# Создание ReAct-агента


Создание агента с помощью `create_react_agent()` из **LangChain**.

Затем оборачиваем агента в `AgentExecutor` — для управления запуском, количеством итераций и логированием.

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

# Создание агента
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Оборачивание агента в Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,               # Вывод промежуточных логов
    handle_parsing_errors=True, # Автоматическая обработка ошибок парсинга (не падать при невалидном ответе)
    max_iterations=5,
    return_intermediate_steps=False,
    output_key="output"
)

print("✅ Агент создан и готов к запуску")

✅ Агент создан и готов к запуску


# Тестирование агента


Протестируем агента на вопросе, где ему понадобится:

* подумать
* при необходимости — использовать инструмент поиска
* вернуть финальный ответ в указанном формате

Цель — добиться от агента правильной последовательности действий, включая:
* корректный выбор инструмента (например, Search)
* осмысленный запрос в Action Input
* корректное извлечение фактов из Observation
* финальный ответ без обрыва на середине

In [ ]:
# Пробный запуск агента
response = agent_executor.invoke({
    "input": "Who is head of development of the GigaChat?"
})

print("\n Ответ агента:")
print(response.get("output"))



> Entering new AgentExecutor chain...


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:2412: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Thought: Yes
Action: Search
Action Input: head of development of GigaChat
ObservGigaChat is a generative artificial intelligence chatbot developed by the Russian financial services corporation Sberbank. It was launched in a closed testing mode in April 2023 and is positioned as a Russian alternative to ChatGPT. [1][2] Mar 13, 2025 · Based on GigaChat 2.0, companies will be able to create more productive autonomous assistants (AI agents) that can reason and solve complex, multi-component problems on their own. Mar 13, 2025 · According to Andrey Belevtsev, Senior Vice President and Head of the Technological Development Unit at Sberbank, " GigaChat 2.0 is not just an increase in metrics and technical characteristics, but a significant step in the development of Russian-speaking large language models (LLM)." Now, as part of the Sberbank artificial intelligence access software interface for the GigaChat API business, two models are available for companies and developers to create their own 

/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:2412: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Final Answer: The head of development of GigaChat is Andrey Belevtsev,

> Finished chain.

 Ответ агента:
The head of development of GigaChat is Andrey Belevtsev,
